In [74]:
import io
import numpy as np
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
from transformers import AutoModel, AutoTokenizer
import onnx
import onnxruntime

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()



In [73]:
model = AutoModel.from_pretrained('/home/spacemanidol/Desktop/CA/models/base-none-nq/query_model')

In [75]:
tokenizer = AutoTokenizer.from_pretrained('/home/spacemanidol/Desktop/CA/models/base-none-nq/query_model')

In [76]:
batch_size = 1    # just a random number
max_seq_length = 32
model.eval()
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [83]:
text = "where was lincoln born?"
encoded = tokenizer(text, return_tensors="pt", padding='max_length', max_length=32)
encoded.pop('token_type_ids')
encoded

{'input_ids': tensor([[ 101, 2073, 2001, 5367, 2141, 1029,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}

In [105]:
outputs = model(**encoded)
outputs['last_hidden_state'][0]

tensor([[-0.0870,  0.3296,  0.0355,  ...,  0.1947,  0.2570,  0.1421],
        [-0.7133, -0.1584,  0.0356,  ..., -0.2093,  0.0280,  0.0826],
        [-0.2436,  0.0579,  0.2785,  ..., -0.1361,  0.1301,  0.2264],
        ...,
        [-0.1396,  0.3695,  0.1416,  ...,  0.1990,  0.1762, -0.0292],
        [-0.1493,  0.3359,  0.1495,  ...,  0.2013,  0.1958,  0.0387],
        [-0.1369,  0.3679,  0.1787,  ...,  0.1477,  0.1959,  0.0619]],
       grad_fn=<SelectBackward>)

In [106]:
ort_session = onnxruntime.InferenceSession("nq-12-direct/query-encoder.onnx")
ort_inputs = {
        'input_ids':  encoded['input_ids'].cpu().reshape(1, max_seq_length).numpy(),
        'attention_mask': encoded['attention_mask'].reshape(1, max_seq_length).numpy(),
    }
ort_outs = ort_session.run(None, ort_inputs)
ort_outs[0][:,0]

array([[-8.69746208e-02,  3.29582453e-01,  3.54542509e-02,
         2.13683508e-02,  3.18207502e-01,  5.90035975e-01,
         3.76429707e-01,  1.90181389e-01, -5.42121902e-02,
        -1.92273259e-01, -1.26699165e-01, -4.68925178e-01,
        -1.54819682e-01,  2.53729343e-01, -2.34477352e-02,
        -9.45501626e-02, -5.29869087e-02,  1.97173744e-01,
        -1.66212007e-01,  1.24779493e-01, -2.70028681e-01,
        -3.44054312e-01,  2.09206715e-02,  2.18795538e-01,
         2.60815889e-01, -2.32247353e-01,  5.01327962e-03,
        -1.79565087e-01, -1.75962418e-01,  8.92373174e-02,
        -1.91611961e-01, -9.91489738e-02,  2.54330844e-01,
        -3.60694945e-01, -3.76306415e-01, -1.38661817e-01,
        -4.58164960e-02, -6.51936457e-02,  2.87819862e-01,
         2.80486703e-01, -1.39322922e-01, -2.35532835e-01,
         7.28404969e-02, -2.43004113e-01, -1.94266945e-01,
        -5.39015979e-02, -2.91696846e-01,  1.30898818e-01,
         1.14240207e-01, -2.92063892e-01,  1.00555509e-0

In [104]:
ort_session = onnxruntime.InferenceSession("nq-6-direct/query-encoder.onnx")
ort_inputs = {
        'input_ids':  encoded['input_ids'].cpu().reshape(1, max_seq_length).numpy(),
        'attention_mask': encoded['attention_mask'].reshape(1, max_seq_length).numpy(),
    }
ort_outs = ort_session.run(None, ort_inputs)
ort_outs[0][:,0]

array([[-1.06191480e+00,  6.77800238e-01, -2.25780219e-01,
         2.18136311e-02, -2.62686342e-01,  2.12000430e-01,
         2.05599144e-01,  2.63661444e-02, -7.14580894e-01,
        -1.55631080e-01, -2.28125602e-01,  2.33234949e-02,
        -6.57021880e-01,  1.88758761e-01,  1.76468760e-01,
         2.08310723e-01,  7.29167536e-02, -5.60141385e-01,
        -5.45580797e-02,  1.35522038e-02, -6.69487357e-01,
        -4.14608210e-01, -7.55360276e-02, -5.16542554e-01,
        -2.03416571e-02, -5.23782134e-01, -1.38682112e-01,
        -1.33681893e-01, -1.05729029e-01,  6.82076275e-01,
        -4.61982280e-01, -1.83635615e-02,  2.60436326e-01,
        -1.29011005e-01, -6.50261104e-01, -2.50722080e-01,
        -9.85088795e-02, -5.03367722e-01,  1.02768019e-01,
         3.62493873e-01, -1.77948877e-01, -7.11698651e-01,
        -5.86913973e-02,  3.57935578e-01, -5.54007351e-01,
        -1.21120408e-01, -1.74600720e+00, -1.26633272e-01,
         4.67982233e-01,  3.33609909e-01,  1.99522674e-0

In [20]:
# Input to the model
input_ids = torch.randint(30000,(1,32))
attention_mask = torch.ones(1, 32)
x = {'input_ids': input_ids, 'attention_mask': attention_mask}

In [68]:
# Input to the model
input_ids = torch.randint(30000,(1,32))
attention_mask = torch.ones(1, 32,dtype=torch.int64)
x = {'input_ids': input_ids, 'attention_mask': attention_mask}
outputs = model(input_ids,attention_mask)
outputs
outputs['last_hidden_state'].shape

torch.Size([1, 32, 768])

In [69]:
torch.onnx.export(model,               # model being run
                  (input_ids,attention_mask), # model input (or a tuple for multiple inputs)
                  "query_encoder.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input_ids','attention_mask'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [72]:
onnx_model = onnx.load("query_encoder.onnx")
print(onnx.checker.check_model(onnx_model))
input_ids = torch.randint(30000,(1,32))
attention_mask = torch.ones(1, 32,dtype=torch.int64)
print(attention_mask)
torch_out = model(input_ids,attention_mask)
ort_session = onnxruntime.InferenceSession("query_encoder.onnx")
input_ids_numpy = input_ids.numpy()
ort_inputs = {
        'input_ids':  input_ids.cpu().reshape(1, max_seq_length).numpy(),
        'attention_mask': attention_mask.reshape(1, max_seq_length).numpy(),
    }
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)
# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(torch_out['last_hidden_state'].detach().numpy(), ort_outs[0], rtol=1e-03, atol=1e-06)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

None
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])
[array([[[0.3451308 , 0.1831994 , 0.18901086, ..., 0.18805812,
         0.13006443, 0.55217695],
        [0.45257404, 0.16564843, 0.27069584, ..., 0.10831616,
         0.1765618 , 0.6336158 ],
        [0.4925477 , 0.11632142, 0.20679797, ..., 0.18854517,
         0.10343742, 0.8939595 ],
        ...,
        [0.5138854 , 0.35133833, 0.24877475, ..., 0.11686493,
         0.14266187, 0.6531461 ],
        [0.4800297 , 0.31081295, 0.25406474, ..., 0.08241704,
         0.19758047, 0.7475122 ],
        [0.2500318 , 0.29603854, 0.21344765, ..., 0.14953123,
         0.16796203, 0.70189834]]], dtype=float32), array([[-5.96702099e-01,  3.18771414e-02,  9.83935595e-01,
         3.06029409e-01, -7.65705705e-01,  5.93425147e-02,
         4.35969353e-01,  1.23715535e-01,  9.79555786e-01,
        -9.10714269e-01,  8.37417483e-01, -8.99365366e-01,
         9.83341694e-01, -9.1075581

In [ ]:
ort_inputs = {'input_ids':  batch['input_ids'].cpu().reshape(1, max_seq_length).numpy(),'attention_mask': batch['attention_mask'].cpu().reshape(1, max_seq_length).numpy(),}
ort_session.run(None, ort_inputs)

In [62]:
# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(torch_out['last_hidden_state'].detach().numpy(), ort_outs[0], rtol=1e-03, atol=1e-06)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [65]:
model_fp32 = "query_encoder.onnx"
model_quant = "query_encoder_quant.onnx"
quantized_model = quantize_dynamic(model_fp32, model_quant)

AttributeError: module 'onnx.shape_inference' has no attribute 'infer_shapes_path'

In [ ]:
trainer = Trainer(
        model=model,
        model_state_path=model_path,
        recipe=None,
        recipe_args=None,
        teacher=None,
    )
applied = trainer.apply_manager(epoch=math.inf, checkpoint=None)